# Comprehensive DuckDB Analysis

## Carrier Invoice Dataset Analysis

This notebook provides a comprehensive analysis of the DuckDB file generated by our ClickHouse extraction pipeline.

### Analysis Scope:
1. **Database Structure** - Tables, schemas, columns, data types
2. **Data Statistics** - Record counts, date ranges, unique identifiers
3. **Data Quality Metrics** - Completeness, null values, data integrity
4. **File Information** - Size, location, extraction metadata

Let's dive in! 🔍

In [1]:
# Import required libraries
import duckdb
import pandas as pd
import numpy as np
import os
from datetime import datetime
from pathlib import Path

# Display settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("📊 Libraries imported successfully!")
print(f"🦆 DuckDB version: {duckdb.__version__}")
print(f"🐼 Pandas version: {pd.__version__}")

📊 Libraries imported successfully!
🦆 DuckDB version: 1.3.2
🐼 Pandas version: 2.3.1


In [2]:
# Connect to the DuckDB file
db_path = "../../carrier_invoice_extraction.duckdb"

print("🔗 Connecting to DuckDB Analysis")
print("=" * 40)

try:
    # Check if file exists
    if not os.path.exists(db_path):
        print(f"❌ DuckDB file not found: {db_path}")
        print("💡 Run the pipeline first: poetry run python src/src/dlt_pipeline_examples.py")
        raise FileNotFoundError(f"DuckDB file not found: {db_path}")
    
    # Get file information
    file_stats = os.stat(db_path)
    file_size_mb = file_stats.st_size / (1024 * 1024)
    last_modified = datetime.fromtimestamp(file_stats.st_mtime)
    
    print(f"📁 File Location: {os.path.abspath(db_path)}")
    print(f"💾 File Size: {file_size_mb:.2f} MB")
    print(f"🕒 Last Modified: {last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Connect to DuckDB
    conn = duckdb.connect(db_path)
    print(f"\n✅ Successfully connected to DuckDB!")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    conn = None

🔗 Connecting to DuckDB Analysis
📁 File Location: c:\Users\Gabriel\Documents\gsr_project\gsr_automation\carrier_invoice_extraction.duckdb
💾 File Size: 24.51 MB
🕒 Last Modified: 2025-09-08 19:49:20

✅ Successfully connected to DuckDB!


## 1. Database Structure Analysis

Examining the database schema, tables, and column structure.

In [3]:
# Analyze database structure
print("🏗️ Database Structure Analysis")
print("=" * 35)

if conn is None:
    print("❌ No database connection available")
else:
    try:
        # 1. List all schemas
        print("📋 Available Schemas:")
        schemas = conn.execute("SELECT schema_name FROM information_schema.schemata ORDER BY schema_name").fetchall()
        for i, schema in enumerate(schemas, 1):
            print(f"   {i}. {schema[0]}")
        
        # 2. List all tables
        print(f"\n📊 Available Tables:")
        tables = conn.execute("""
            SELECT table_schema, table_name, table_type 
            FROM information_schema.tables 
            WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
            ORDER BY table_schema, table_name
        """).fetchall()
        
        for i, table in enumerate(tables, 1):
            print(f"   {i}. {table[0]}.{table[1]} ({table[2]})")
        
        # 3. Focus on the main carrier invoice table
        main_table = "carrier_invoice_extraction.carrier_invoice_data"
        print(f"\n🎯 Main Table: {main_table}")
        
        # Check if main table exists
        table_exists = conn.execute(f"""
            SELECT COUNT(*) FROM information_schema.tables 
            WHERE table_schema = 'carrier_invoice_data' 
            AND table_name = 'carrier_invoice_data'
        """).fetchone()[0]
        
        if table_exists == 0:
            print(f"❌ Main table {main_table} not found")
            print("💡 Available tables listed above")
        else:
            print(f"✅ Main table {main_table} found")
            
            # Get basic table info
            row_count = conn.execute(f"SELECT COUNT(*) FROM {main_table}").fetchone()[0]
            print(f"📊 Total Records: {row_count:,}")
    
    except Exception as e:
        print(f"❌ Structure analysis failed: {e}")

🏗️ Database Structure Analysis
📋 Available Schemas:
   1. information_schema
   2. main
   3. main
   4. main
   5. pg_catalog

📊 Available Tables:
   1. main.carrier_invoice_data (BASE TABLE)

🎯 Main Table: carrier_invoice_extraction.carrier_invoice_data
❌ Main table carrier_invoice_extraction.carrier_invoice_data not found
💡 Available tables listed above


In [4]:
# Detailed column analysis
print("📋 Column Structure Analysis")
print("=" * 30)

if conn is None:
    print("❌ No database connection available")
else:
    try:
        main_table = "carrier_invoice_extraction.carrier_invoice_data"

        # Get column information
        columns_info = conn.execute(f"""
            SELECT 
                column_name,
                data_type,
                is_nullable,
                column_default,
                ordinal_position
            FROM information_schema.columns 
            WHERE table_schema = 'carrier_invoice_data' 
            AND table_name = 'carrier_invoice_data'
            ORDER BY ordinal_position
        """).fetchall()

        if not columns_info:
            print(f"❌ No column information found for {main_table}")
        else:
            print(f"📊 Total Columns: {len(columns_info)}")
            print(f"\n📋 Column Details:")

            # Create DataFrame for better display
            df_columns = pd.DataFrame(columns_info, columns=[
                'Column Name', 'Data Type', 'Nullable', 'Default', 'Position'
            ])

            # Show first 20 columns
            print(f"\n🔍 First 20 Columns:")
            print(df_columns.head(20).to_string(index=False))

            if len(df_columns) > 20:
                print(f"\n... and {len(df_columns) - 20} more columns")

            # Data type summary
            print(f"\n📊 Data Type Summary:")
            type_counts = df_columns['Data Type'].value_counts()
            for dtype, count in type_counts.items():
                print(f"   {dtype}: {count} columns")

            # Key business columns
            key_columns = [
                'invoice_number', 'invoice_date', 'invoice_amount', 'invoice_currency_code',
                'tracking_number', 'sender_company_name', 'receiver_company_name',
                'charge_description', 'net_amount', 'import_time', '_extracted_at'
            ]

            print(f"\n💼 Key Business Columns:")
            for col in key_columns:
                col_info = df_columns[df_columns['Column Name'] == col]
                if not col_info.empty:
                    dtype = col_info.iloc[0]['Data Type']
                    print(f"   ✅ {col:25} | {dtype}")
                else:
                    print(f"   ❌ {col:25} | Not found")

    except Exception as e:
        print(f"❌ Column analysis failed: {e}")

📋 Column Structure Analysis
❌ No column information found for carrier_invoice_extraction.carrier_invoice_data


## 2. Data Statistics Analysis

Examining record counts, date ranges, and key identifier statistics.

In [5]:
# Basic data statistics
print("📊 Data Statistics Analysis")
print("=" * 30)

if conn is None:
    print("❌ No database connection available")
else:
    try:
        main_table = "carrier_invoice_extraction.carrier_invoice_data"

        # 1. Basic record counts
        print("📈 Record Count Analysis:")
        total_records = conn.execute(f"SELECT COUNT(*) FROM {main_table}").fetchone()[0]
        print(f"   Total Records: {total_records:,}")

        # 2. Date range analysis
        print(f"\n📅 Date Range Analysis:")

        # Check invoice_date
        try:
            date_stats = conn.execute(f"""
                SELECT 
                    MIN(invoice_date) as earliest_invoice,
                    MAX(invoice_date) as latest_invoice,
                    COUNT(DISTINCT invoice_date) as unique_dates
                FROM {main_table}
                WHERE invoice_date IS NOT NULL AND invoice_date != ''
            """).fetchone()

            print(f"   Invoice Date Range: {date_stats[0]} to {date_stats[1]}")
            print(f"   Unique Invoice Dates: {date_stats[2]:,}")
        except Exception as e:
            print(f"   ❌ Invoice date analysis failed: {e}")

        # Check import_time
        try:
            import_stats = conn.execute(f"""
                SELECT 
                    MIN(import_time) as earliest_import,
                    MAX(import_time) as latest_import
                FROM {main_table}
                WHERE import_time IS NOT NULL
            """).fetchone()

            print(f"   Import Time Range: {import_stats[0]} to {import_stats[1]}")
        except Exception as e:
            print(f"   ❌ Import time analysis failed: {e}")

        # Check _extracted_at
        try:
            extract_stats = conn.execute(f"""
                SELECT 
                    MIN(_extracted_at) as earliest_extraction,
                    MAX(_extracted_at) as latest_extraction
                FROM {main_table}
                WHERE _extracted_at IS NOT NULL
            """).fetchone()

            print(f"   Extraction Time Range: {extract_stats[0]} to {extract_stats[1]}")
        except Exception as e:
            print(f"   ❌ Extraction time analysis failed: {e}")

        # 3. Unique identifier counts
        print(f"\n🔑 Unique Identifier Analysis:")

        identifiers = [
            ('invoice_number', 'Invoice Numbers'),
            ('tracking_number', 'Tracking Numbers'),
            ('account_number', 'Account Numbers'),
            ('sender_company_name', 'Sender Companies'),
            ('receiver_company_name', 'Receiver Companies')
        ]

        for col_name, display_name in identifiers:
            try:
                unique_count = conn.execute(f"""
                    SELECT COUNT(DISTINCT {col_name}) 
                    FROM {main_table}
                    WHERE {col_name} IS NOT NULL AND {col_name} != ''
                """).fetchone()[0]

                print(f"   {display_name}: {unique_count:,}")
            except Exception as e:
                print(f"   ❌ {display_name}: Analysis failed")

    except Exception as e:
        print(f"❌ Statistics analysis failed: {e}")

📊 Data Statistics Analysis
📈 Record Count Analysis:
   Total Records: 144,910

📅 Date Range Analysis:
   Invoice Date Range: 2025-08-09 to 2025-08-09
   Unique Invoice Dates: 1
   Import Time Range: 2025-08-10 10:53:22.183318+08:00 to 2025-08-11 20:05:49.820707+08:00
   Extraction Time Range: 2025-08-19 07:51:11.929653+08:00 to 2025-08-19 07:52:05.785236+08:00

🔑 Unique Identifier Analysis:
   Invoice Numbers: 178
   Tracking Numbers: 32,384
   Account Numbers: 178
   Sender Companies: 1,450
   Receiver Companies: 24,212


In [6]:
query = f"""SELECT 
    tracking_number,
    invoice_date,
    shipment_date,
    transaction_date,
    STRFTIME(
        COALESCE(
            TRY_STRPTIME(invoice_date, '%Y-%m-%d'),
            TRY_STRPTIME(invoice_date, '%m/%d/%Y')
        ),
        '%Y-%m-%d'
    ) AS standardized_invoice_date
FROM {main_table}
WHERE COALESCE(
    TRY_STRPTIME(transaction_date, '%Y-%m-%d'),
    TRY_STRPTIME(transaction_date, '%m/%d/%Y')
) >= CURRENT_DATE - INTERVAL '7 days';
        """

query_1 = f"""
    select 
    COUNT(CASE WHEN shipment_date IS NULL THEN 1 END)  as null_count,
    COUNT(CASE WHEN shipment_date IS NOT NULL THEN 1 END) AS not_null_count
    from {main_table}
    ;
"""

query_3 = f"""
    select tracking_number,transaction_date, invoice_date, 
    from {main_table}
    
    ;
"""

query_4 = f"""
    SELECT COUNT(DISTINCT tracking_number) AS tracking_count
FROM {main_table}
WHERE DATE(
        CASE 
            WHEN transaction_date LIKE '%/%/%' 
                 THEN STRFTIME('%Y-%m-%d', STRPTIME(transaction_date, '%m/%d/%Y'))
            WHEN transaction_date LIKE '%-%-%' 
                 THEN STRFTIME('%Y-%m-%d', STRPTIME(transaction_date, '%Y-%m-%d'))
        END
      ) >= CURRENT_DATE - INTERVAL 20 DAY;
    ;
"""

query_5 = f"""
    SELECT COUNT(DISTINCT tracking_number) AS tracking_count
FROM {main_table}
    ;
"""

# Run query and directly convert to a Pandas DataFrame
df = conn.execute(query_3).df()

print(df)
# df.to_csv("shipment_invoice_date.csv", index=False)

           tracking_number transaction_date invoice_date
0       1Z2F6Y330335239010       2025-08-04   2025-08-09
1       1Z2F6Y330307908224       2025-07-31   2025-08-09
2                                2025-08-09   2025-08-09
3       1Z2F6Y330337780896       2025-08-05   2025-08-09
4       1Z2F6Y330337780896       2025-08-05   2025-08-09
...                    ...              ...          ...
144905  1ZTT55371394846134       2025-08-06   2025-08-09
144906  1ZTT55371394846134       2025-08-06   2025-08-09
144907  1ZTT55370397867107       2025-08-06   2025-08-09
144908  1ZTT55370398542152       2025-08-06   2025-08-09
144909  1ZTT55370391395440       2025-08-05   2025-08-09

[144910 rows x 3 columns]


In [7]:
columns_info = conn.execute(
    f"""
            SELECT 
                column_name,
                data_type,
                is_nullable,
                column_default,
                ordinal_position
            FROM information_schema.columns 
            WHERE table_schema = 'carrier_invoice_data' 
            AND table_name = 'carrier_invoice_data'
            ORDER BY ordinal_position
        """
).fetchall()

print(columns_info)

[]


In [8]:
# Currency and amount analysis
print("💰 Currency & Amount Analysis")
print("=" * 32)

if conn is None:
    print("❌ No database connection available")
else:
    try:
        main_table = "carrier_invoice_extraction.carrier_invoice_data"
        
        # Currency distribution
        print("💱 Currency Distribution:")
        currencies = conn.execute(f"""
            SELECT 
                invoice_currency_code,
                COUNT(*) as count,
                ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
            FROM {main_table}
            WHERE invoice_currency_code IS NOT NULL AND invoice_currency_code != ''
            GROUP BY invoice_currency_code
            ORDER BY count DESC
            LIMIT 10
        """).fetchall()
        
        for curr in currencies:
            print(f"   {curr[0]:5} | {curr[1]:,} records ({curr[2]}%)")
        
        # Amount statistics
        print(f"\n💵 Invoice Amount Statistics:")
        try:
            amount_stats = conn.execute(f"""
                SELECT 
                    COUNT(*) as total_with_amounts,
                    ROUND(AVG(CAST(invoice_amount AS DECIMAL)), 2) as avg_amount,
                    ROUND(MIN(CAST(invoice_amount AS DECIMAL)), 2) as min_amount,
                    ROUND(MAX(CAST(invoice_amount AS DECIMAL)), 2) as max_amount,
                    ROUND(SUM(CAST(invoice_amount AS DECIMAL)), 2) as total_amount
                FROM {main_table}
                WHERE invoice_amount IS NOT NULL 
                AND invoice_amount != ''
                AND TRY_CAST(invoice_amount AS DECIMAL) IS NOT NULL
                AND CAST(invoice_amount AS DECIMAL) >= 0
            """).fetchone()
            
            print(f"   Records with Valid Amounts: {amount_stats[0]:,}")
            print(f"   Average Amount: ${amount_stats[1]:,.2f}")
            print(f"   Minimum Amount: ${amount_stats[2]:,.2f}")
            print(f"   Maximum Amount: ${amount_stats[3]:,.2f}")
            print(f"   Total Amount: ${amount_stats[4]:,.2f}")
            
        except Exception as e:
            print(f"   ❌ Amount statistics failed: {e}")
        
        # Top charge types
        print(f"\n📋 Top Charge Types:")
        try:
            charges = conn.execute(f"""
                SELECT 
                    charge_description,
                    COUNT(*) as count
                FROM {main_table}
                WHERE charge_description IS NOT NULL AND charge_description != ''
                GROUP BY charge_description
                ORDER BY count DESC
                LIMIT 10
            """).fetchall()
            
            for charge in charges:
                print(f"   {charge[0][:40]:40} | {charge[1]:,} records")
                
        except Exception as e:
            print(f"   ❌ Charge analysis failed: {e}")
    
    except Exception as e:
        print(f"❌ Currency analysis failed: {e}")

💰 Currency & Amount Analysis
💱 Currency Distribution:
❌ Currency analysis failed: Catalog Error: Table with name carrier_invoice_data does not exist!
Did you mean "carrier_invoice_extraction.carrier_invoice_data"?

LINE 6:             FROM carrier_invoice_extraction.carrier_invoice_data
                         ^


## 3. Data Quality Metrics

Analyzing data completeness, null values, and data integrity.

In [9]:
# Data quality analysis
print("🔍 Data Quality Analysis")
print("=" * 25)

if conn is None:
    print("❌ No database connection available")
else:
    try:
        main_table = "carrier_invoice_extraction.carrier_invoice_data"
        
        # Get total record count for percentage calculations
        total_records = conn.execute(f"SELECT COUNT(*) FROM {main_table}").fetchone()[0]
        
        # Key business columns to analyze
        key_columns = [
            'invoice_number',
            'invoice_date', 
            'invoice_amount',
            'invoice_currency_code',
            'tracking_number',
            'sender_company_name',
            'receiver_company_name',
            'charge_description',
            'net_amount'
        ]
        
        print(f"📊 Data Completeness Analysis (Total Records: {total_records:,}):")
        print()
        
        completeness_data = []
        
        for column in key_columns:
            try:
                # Count non-null, non-empty values
                valid_count = conn.execute(f"""
                    SELECT COUNT(*) 
                    FROM {main_table}
                    WHERE {column} IS NOT NULL AND {column} != ''
                """).fetchone()[0]
                
                # Count null values
                null_count = conn.execute(f"""
                    SELECT COUNT(*) 
                    FROM {main_table}
                    WHERE {column} IS NULL
                """).fetchone()[0]
                
                # Count empty string values
                empty_count = conn.execute(f"""
                    SELECT COUNT(*) 
                    FROM {main_table}
                    WHERE {column} = ''
                """).fetchone()[0]
                
                completeness_pct = (valid_count / total_records) * 100
                missing_count = null_count + empty_count
                
                completeness_data.append({
                    'Column': column,
                    'Valid Count': valid_count,
                    'Missing Count': missing_count,
                    'Completeness %': completeness_pct
                })
                
                # Color coding for completeness
                if completeness_pct >= 95:
                    status = "✅ Excellent"
                elif completeness_pct >= 80:
                    status = "🟡 Good"
                elif completeness_pct >= 50:
                    status = "🟠 Fair"
                else:
                    status = "❌ Poor"
                
                print(f"   {column:25} | {valid_count:>8,} valid | {missing_count:>8,} missing | {completeness_pct:>6.1f}% | {status}")
                
            except Exception as e:
                print(f"   {column:25} | ❌ Analysis failed: {e}")
        
        # Summary statistics
        if completeness_data:
            df_completeness = pd.DataFrame(completeness_data)
            avg_completeness = df_completeness['Completeness %'].mean()
            
            print(f"\n📈 Data Quality Summary:")
            print(f"   Average Completeness: {avg_completeness:.1f}%")
            print(f"   Columns with >95% completeness: {len(df_completeness[df_completeness['Completeness %'] >= 95])}")
            print(f"   Columns with <50% completeness: {len(df_completeness[df_completeness['Completeness %'] < 50])}")
    
    except Exception as e:
        print(f"❌ Data quality analysis failed: {e}")

🔍 Data Quality Analysis
❌ Data quality analysis failed: Catalog Error: Table with name carrier_invoice_data does not exist!
Did you mean "carrier_invoice_extraction.carrier_invoice_data"?

LINE 1: SELECT COUNT(*) FROM carrier_invoice_extraction.carrier_invoice_data
                             ^


In [10]:
# Sample data analysis
print("📋 Sample Data Analysis")
print("=" * 25)

if conn is None:
    print("❌ No database connection available")
else:
    try:
        main_table = "carrier_invoice_extraction.carrier_invoice_data"
        
        # Get sample records with key fields
        print("📊 Sample Records (Key Fields):")
        sample_query = f"""
            SELECT 
                invoice_number,
                invoice_date,
                invoice_amount,
                invoice_currency_code,
                tracking_number,
                sender_company_name,
                receiver_company_name,
                charge_description,
                net_amount,
                import_time
            FROM {main_table}
            ORDER BY import_time DESC
            LIMIT 5
        """
        
        sample_df = conn.execute(sample_query).df()
        
        if not sample_df.empty:
            # Display sample with better formatting
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', 30)
            
            print(sample_df.to_string(index=False))
            
            print(f"\n✅ Sample data shows {len(sample_df)} recent records")
        else:
            print("❌ No sample data available")
        
        # Data validation checks
        print(f"\n🔍 Data Validation Checks:")
        
        # Check for duplicate invoice numbers
        try:
            duplicate_invoices = conn.execute(f"""
                SELECT COUNT(*) 
                FROM (
                    SELECT invoice_number, COUNT(*) as cnt
                    FROM {main_table}
                    WHERE invoice_number IS NOT NULL AND invoice_number != ''
                    GROUP BY invoice_number
                    HAVING COUNT(*) > 1
                ) duplicates
            """).fetchone()[0]
            
            print(f"   Duplicate Invoice Numbers: {duplicate_invoices:,}")
        except Exception as e:
            print(f"   ❌ Duplicate check failed: {e}")
        
        # Check for invalid amounts
        try:
            invalid_amounts = conn.execute(f"""
                SELECT COUNT(*) 
                FROM {main_table}
                WHERE invoice_amount IS NOT NULL 
                AND invoice_amount != ''
                AND TRY_CAST(invoice_amount AS DECIMAL) IS NULL
            """).fetchone()[0]
            
            print(f"   Invalid Amount Values: {invalid_amounts:,}")
        except Exception as e:
            print(f"   ❌ Amount validation failed: {e}")
        
        # Check for future dates
        try:
            future_dates = conn.execute(f"""
                SELECT COUNT(*) 
                FROM {main_table}
                WHERE invoice_date > CURRENT_DATE
            """).fetchone()[0]
            
            print(f"   Future Invoice Dates: {future_dates:,}")
        except Exception as e:
            print(f"   ❌ Date validation failed: {e}")
    
    except Exception as e:
        print(f"❌ Sample data analysis failed: {e}")

📋 Sample Data Analysis
📊 Sample Records (Key Fields):
❌ Sample data analysis failed: Catalog Error: Table with name carrier_invoice_data does not exist!
Did you mean "carrier_invoice_extraction.carrier_invoice_data"?

LINE 13:             FROM carrier_invoice_extraction.carrier_invoice_data
                          ^


## 4. File Information & Comparison

Detailed file information and comparison with source data.

In [11]:
# Comprehensive file information
print("📁 File Information Analysis")
print("=" * 30)

if conn is None:
    print("❌ No database connection available")
else:
    try:
        # File system information
        file_stats = os.stat(db_path)
        file_size_bytes = file_stats.st_size
        file_size_mb = file_size_bytes / (1024 * 1024)
        file_size_gb = file_size_mb / 1024
        
        created_time = datetime.fromtimestamp(file_stats.st_ctime)
        modified_time = datetime.fromtimestamp(file_stats.st_mtime)
        accessed_time = datetime.fromtimestamp(file_stats.st_atime)
        
        print(f"📊 File System Information:")
        print(f"   File Path: {os.path.abspath(db_path)}")
        print(f"   File Size: {file_size_mb:.2f} MB ({file_size_gb:.3f} GB)")
        print(f"   Created: {created_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   Modified: {modified_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   Last Accessed: {accessed_time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Database metadata
        print(f"\n🗄️ Database Metadata:")
        
        main_table = "carrier_invoice_extraction.carrier_invoice_data"
        
        # Get extraction metadata
        try:
            extraction_info = conn.execute(f"""
                SELECT 
                    MIN(_extracted_at) as first_extraction,
                    MAX(_extracted_at) as last_extraction,
                    COUNT(DISTINCT _dlt_load_id) as load_sessions,
                    COUNT(DISTINCT _source_table) as source_tables
                FROM {main_table}
                WHERE _extracted_at IS NOT NULL
            """).fetchone()
            
            print(f"   First Extraction: {extraction_info[0]}")
            print(f"   Last Extraction: {extraction_info[1]}")
            print(f"   Load Sessions: {extraction_info[2]}")
            print(f"   Source Tables: {extraction_info[3]}")
            
        except Exception as e:
            print(f"   ❌ Extraction metadata failed: {e}")
        
        # DLT metadata
        try:
            dlt_info = conn.execute(f"""
                SELECT 
                    COUNT(DISTINCT _dlt_load_id) as unique_loads,
                    COUNT(DISTINCT _dlt_id) as unique_dlt_ids
                FROM {main_table}
            """).fetchone()
            
            print(f"   DLT Load IDs: {dlt_info[0]}")
            print(f"   DLT Record IDs: {dlt_info[1]}")
            
        except Exception as e:
            print(f"   ❌ DLT metadata failed: {e}")
        
        # Storage efficiency
        total_records = conn.execute(f"SELECT COUNT(*) FROM {main_table}").fetchone()[0]
        total_columns = len(conn.execute(f"DESCRIBE {main_table}").fetchall())
        
        bytes_per_record = file_size_bytes / total_records if total_records > 0 else 0
        
        print(f"\n📈 Storage Efficiency:")
        print(f"   Records: {total_records:,}")
        print(f"   Columns: {total_columns}")
        print(f"   Bytes per Record: {bytes_per_record:.2f}")
        print(f"   Estimated CSV Size: {(bytes_per_record * total_records * 1.5) / (1024*1024):.2f} MB")
    
    except Exception as e:
        print(f"❌ File information analysis failed: {e}")

📁 File Information Analysis
📊 File System Information:
   File Path: c:\Users\Gabriel\Documents\gsr_project\gsr_automation\carrier_invoice_extraction.duckdb
   File Size: 24.51 MB (0.024 GB)
   Created: 2025-09-08 20:13:17
   Modified: 2025-09-08 19:49:20
   Last Accessed: 2025-09-08 20:33:25

🗄️ Database Metadata:
   ❌ Extraction metadata failed: Catalog Error: Table with name carrier_invoice_data does not exist!
Did you mean "carrier_invoice_extraction.carrier_invoice_data"?

LINE 7:                 FROM carrier_invoice_extraction.carrier_invoice_data
                             ^
   ❌ DLT metadata failed: Catalog Error: Table with name carrier_invoice_data does not exist!
Did you mean "carrier_invoice_extraction.carrier_invoice_data"?

LINE 5:                 FROM carrier_invoice_extraction.carrier_invoice_data
                             ^
❌ File information analysis failed: Catalog Error: Table with name carrier_invoice_data does not exist!
Did you mean "carrier_invoice_extracti

In [ ]:
# Compare with ClickHouse source (if accessible)
print("🔄 ClickHouse Source Comparison")
print("=" * 35)

try:
    import clickhouse_connect
    
    print("📡 Attempting to connect to ClickHouse source...")
    
    # Connect to ClickHouse
    ch_client = clickhouse_connect.get_client(
        host="pgy8egpix3.us-east-1.aws.clickhouse.cloud",
        port=8443,
        username="gabriellapuz",
        password="PTN.776)RR3s",
        database="peerdb",
        secure=True,
    )
    
    print("✅ Connected to ClickHouse successfully!")
    
    # Compare record counts
    ch_table = "carrier_carrier_invoice_original_flat_ups"
    ch_total = ch_client.command(f"SELECT COUNT(*) FROM {ch_table}")
    
    if conn:
        duck_total = conn.execute(f"SELECT COUNT(*) FROM carrier_invoice_extraction.carrier_invoice_data").fetchone()[0]
        
        print(f"\n📊 Record Count Comparison:")
        print(f"   ClickHouse Source: {ch_total:,} records")
        print(f"   DuckDB Extract: {duck_total:,} records")
        
        coverage_pct = (duck_total / ch_total) * 100 if ch_total > 0 else 0
        print(f"   Coverage: {coverage_pct:.2f}%")
        
        if coverage_pct < 100:
            missing_records = ch_total - duck_total
            print(f"   Missing Records: {missing_records:,}")
            print(f"   💡 This is expected for incremental extractions")
    
    # Compare latest data timestamps
    try:
        ch_latest = ch_client.command(f"SELECT MAX(import_time) FROM {ch_table}")
        
        if conn:
            duck_latest = conn.execute(f"SELECT MAX(import_time) FROM carrier_invoice_extraction.carrier_invoice_data").fetchone()[0]
            
            print(f"\n📅 Latest Data Comparison:")
            print(f"   ClickHouse Latest: {ch_latest}")
            print(f"   DuckDB Latest: {duck_latest}")
            
            if str(ch_latest) == str(duck_latest):
                print(f"   ✅ Data is up to date")
            else:
                print(f"   🟡 Data may need updating")
    
    except Exception as e:
        print(f"   ❌ Timestamp comparison failed: {e}")
    
    ch_client.close()
    
except ImportError:
    print("❌ ClickHouse library not available for comparison")
    print("💡 Install with: pip install clickhouse-connect")
except Exception as e:
    print(f"❌ ClickHouse comparison failed: {e}")
    print("💡 This is normal if ClickHouse is not accessible")

🔄 ClickHouse Source Comparison
📡 Attempting to connect to ClickHouse source...


In [ ]:
# Generate comprehensive summary report
print("📋 Comprehensive Analysis Summary")
print("=" * 40)

if conn is None:
    print("❌ No database connection available for summary")
else:
    try:
        main_table = "carrier_invoice_extraction.carrier_invoice_data"
        
        # Collect key metrics
        total_records = conn.execute(f"SELECT COUNT(*) FROM {main_table}").fetchone()[0]
        total_columns = len(conn.execute(f"DESCRIBE {main_table}").fetchall())
        
        # File info
        file_size_mb = os.path.getsize(db_path) / (1024 * 1024)
        
        # Date ranges
        try:
            date_range = conn.execute(f"""
                SELECT 
                    MIN(invoice_date) as earliest,
                    MAX(invoice_date) as latest
                FROM {main_table}
                WHERE invoice_date IS NOT NULL AND invoice_date != ''
            """).fetchone()
        except:
            date_range = ("Unknown", "Unknown")
        
        # Unique identifiers
        try:
            unique_invoices = conn.execute(f"""
                SELECT COUNT(DISTINCT invoice_number) 
                FROM {main_table}
                WHERE invoice_number IS NOT NULL AND invoice_number != ''
            """).fetchone()[0]
        except:
            unique_invoices = "Unknown"
        
        print(f"🎯 DATASET OVERVIEW:")
        print(f"   📊 Total Records: {total_records:,}")
        print(f"   📋 Total Columns: {total_columns}")
        print(f"   💾 File Size: {file_size_mb:.2f} MB")
        print(f"   📅 Date Range: {date_range[0]} to {date_range[1]}")
        print(f"   🔑 Unique Invoices: {unique_invoices:,}" if isinstance(unique_invoices, int) else f"   🔑 Unique Invoices: {unique_invoices}")
        
        print(f"\n✅ ANALYSIS COMPLETE:")
        print(f"   📁 DuckDB file successfully analyzed")
        print(f"   🔍 Data structure documented")
        print(f"   📊 Statistics calculated")
        print(f"   🎯 Quality metrics assessed")
        
        print(f"\n💡 NEXT STEPS:")
        print(f"   📊 Use the CSV export notebook to extract data")
        print(f"   📈 Perform business analysis on the dataset")
        print(f"   🔄 Schedule regular pipeline updates")
        print(f"   📋 Monitor data quality over time")
    
    except Exception as e:
        print(f"❌ Summary generation failed: {e}")

# Close connection
if conn:
    conn.close()
    print(f"\n🔒 Database connection closed")

📋 Comprehensive Analysis Summary
🎯 DATASET OVERVIEW:
   📊 Total Records: 935,481
   📋 Total Columns: 263
   💾 File Size: 203.51 MB
   📅 Date Range: 2025-07-05 to 2025-08-23
   🔑 Unique Invoices: 1,126

✅ ANALYSIS COMPLETE:
   📁 DuckDB file successfully analyzed
   🔍 Data structure documented
   📊 Statistics calculated
   🎯 Quality metrics assessed

💡 NEXT STEPS:
   📊 Use the CSV export notebook to extract data
   📈 Perform business analysis on the dataset
   🔄 Schedule regular pipeline updates
   📋 Monitor data quality over time

🔒 Database connection closed
